In [3]:
import urllib2
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np

In [4]:
rooturl = 'http://www.assemblee-nationale.fr/14/cri/'

In [5]:
year2016 = '2015-2016/'
url2016 = rooturl + year2016
page2016 = urllib2.urlopen(url2016)

In [6]:
soup2016 = BeautifulSoup(page2016, 'lxml')

In [7]:
def get_questions_reponses(tag):
    return tag.name == 'td' and re.compile('Questions au Gouvernement').search(tag.text) and re.compile('Version provisoire').search(tag.text) is None

In [8]:
table = soup2016.find_all(get_questions_reponses)

In [24]:
urlsCR_dates = [(url2016 + x.a.get('href'), x.parent.find_previous_sibling('tr').text.strip()) for x in table]

In [10]:
soups = [(BeautifulSoup(urllib2.urlopen(u[0]), "lxml"), u[1]) for u in urlsCR_dates]

In [11]:
def extract_from_par(title, par):
    speech={}
    speaker = par.b.string
    reactions = [d.string for d in par.find_all('i', string=re.compile(r"^\((.*)\)$") )]
    for italique in par.find_all('i'):
        italique.replaceWith('')
    if 'Mme' in speaker :
        sexe = "F" 
    elif 'M.' in speaker :
        sexe = "M" 
    else :
        sexe = "Unknown" 
    if re.match(r"^"+speaker+",", par.text) != None:
        role = "ministre"
    elif re.match(r"^M. le pr", speaker) != None :
        role = "president"
    else :
        role = "depute"
    all_text = par.text[len(speaker)+1:].strip()
    return {
        "speaker":speaker,
        "sexe":sexe,
        "speech":all_text,
        "reactions": reactions,
        "nb_words":len(all_text.split()),
        "role":role
    }


In [12]:
def interventions(title, soup):
    return [extract_from_par(title, y) for y in soup.find_all('p') if y.b != None]

In [13]:
dic = {}
def fill_dict(p, date):
    questions = p.findAll("h2", class_= "titre2")
    for question in questions:
        title = question.text
        interv = interventions(title, question.find_parent("div", class_= "Point"))
        dic[(title, date)] = interv

In [14]:
for s in soups:
    for q in s[0].findAll("h2", class_= "titre1", text = re.compile(r"^Questions au Gouvernement")):
        fill_dict(q.find_parent("div", class_= "Point"), s[1])

In [21]:
df = pd.DataFrame(list(dic.iteritems()), columns=['Question_tuple', 'Speeche'])
df['Date'] = df['Question_tuple'].apply(lambda x: x[1])
df['Question'] = df['Question_tuple'].apply(lambda x: x[0])
df['Number of interventions'] = df['Speeche'].apply(lambda x: len(x))
df.drop('Question_tuple', axis=1, inplace=True)
df = df[['Question', 'Date', 'Speeche', 'Number of interventions']]
df.reset_index(drop=True, inplace=True)

In [22]:
nb_words={'F':0,'M':0,'Unknown':0}
nb_sexe={'F':0,'M':0,'Unknown':0}
for speech in df['Speeche']:
    if len(speech)>1:
        speech[1]['role']="depute_question"
        depute_principal = speech[1]['speaker']
        for s in speech :
            if s != None and s['role'] == "depute" and s['speaker'] != depute_principal:
                nb_words[s['sexe']]+=s['nb_words']
                nb_sexe[s['sexe']] += 1
            elif s['speaker'] == depute_principal:
                s['role']="depute_question"
for key, value in nb_sexe.iteritems():
    print key +" : "+ str(nb_words[key]/float(value))

Unknown : 3.94495412844
M : 5.40077821012
F : 9.70017035775


In [23]:
echange = dict()
for speech in df['Speeche']:
    for i, s in enumerate(speech) :
        if s['role']=='depute_question':
            key = s['sexe']
            if key not in echange :
                echange[key]={'total':0,'nb_interruptions':0,'pourcentage_i':0}
            echange[key]['total'] += 1
            if re.match(u'.+…$',s['speech']) != None and i+1 < len(speech)\
            and re.match(u'\D*[!\?]+\D*',speech[i+1]['speech']) != None : 
                echange[key]['nb_interruptions'] += 1
                echange[key]['pourcentage_i'] = echange[key]['nb_interruptions'] / float(echange[key]['total'])
                #echange[key]['Phrases'].append(speech[i+1])

print(echange)  

{'M': {'total': 1311, 'pourcentage_i': 0.07947530864197531, 'nb_interruptions': 103}, 'F': {'total': 373, 'pourcentage_i': 0.05763688760806916, 'nb_interruptions': 20}}


In [27]:
df['list_interlocuteurs'] = df['Speeche'].apply(lambda x: list(set([s['speaker'] for i, s in enumerate(x) if s['role'] not in ['minitre','president','depute_question']])))
df['list_interlocuteurs_F'] = df['Speeche'].apply(lambda x: list(set([s['speaker'] for i, s in enumerate(x)if s['role'] not in ['minitre','president','depute_question'] and s['sexe']=='F'])))
df['list_interlocuteurs_H'] = df['Speeche'].apply(lambda x: list(set([s['speaker'] for i, s in enumerate(x)if s['role'] not in ['minitre','president','depute_question'] and s['sexe']=='M'])))
df['list_interlocuteurs_Unknown'] = df['Speeche'].apply(lambda x: list(set([s['speaker'] for i, s in enumerate(x)if s['role'] not in ['minitre','president','depute_question'] and s['sexe']=='Unknown'])))

nb_i = len(list(set(df['list_interlocuteurs'].sum())))
nb_i_F = len(list(set(df['list_interlocuteurs_F'].sum())))
nb_i_H = len(list(set(df['list_interlocuteurs_H'].sum())))
nb_i_U = len(list(set(df['list_interlocuteurs_Unknown'].sum())))
nb_i_F/float(nb_i)
#df1['list_interlocuteurs'] = df1['list_interlocuteurs'].apply(lambda x : list(set(x)))
#df1['list_interlocuteurs'] = df1['list_interlocuteurs'].apply(lambda x : [i for i in x if 'Mme' in i])

#df1['nb_interlocuteurs'] = df1['list_interlocuteurs'].apply(lambda x : len(x))
#df1['nb_interlocuteurs_F'] = df1['list_interlocuteurs'].apply(lambda x : len([i for i in x if 'Mme' in i]))
#df1['nb_interlocuteurs_H'] = df1['list_interlocuteurs'].apply(lambda x : len([i for i in x if 'M.' in i]))
#df1['perc_interlocuteurs_F'] = df1['nb_interlocuteurs_F']/df1['nb_interlocuteurs']
#df1.mean()

0.2768166089965398

In [20]:
df

,Question,Date,Speeche,Number of interventions
0,Fin de la trêve hivernale,mercredi 30 mars 2016,"[{u'reactions': [], u'sexe': u'M', u'nb_words'...",6
1,Sélection en master,mercredi 30 mars 2016,"[{u'reactions': [], u'sexe': u'M', u'nb_words'...",16
2,Autisme,mercredi 30 mars 2016,"[{u'reactions': [], u'sexe': u'M', u'nb_words'...",12
3,"Mise en œuvre de la loi pour la croissance, l’...",mercredi 30 mars 2016,"[{u'reactions': [], u'sexe': u'M', u'nb_words'...",4
4,Allocation aux adultes handicapéset revenus de...,mercredi 30 mars 2016,"[{u'reactions': [], u'sexe': u'M', u'nb_words'...",9
5,Personnes radicalisées dans les métiers sensibles,mercredi 30 mars 2016,"[{u'reactions': [], u'sexe': u'M', u'nb_words'...",4
6,Dette publique,mercredi 30 mars 2016,"[{u'reactions': [], u'sexe': u'M', u'nb_words'...",26
7,Transparence et lutte contre la corruption,mercredi 30 mars 2016,"[{u'reactions': [], u'sexe': u'M', u'nb_words'...",6
8,Finances locales,mercredi 30 mars 2016,"[{u'reactions': [], u'sexe': u'M', u'nb_words'...",23
9,Garantie jeunes,mercredi 30 mars 2016,"[{u'reactions': [], u'sexe': u'M', u'nb_words'...",10


In [59]:
nb_questions =df['Speeche'].count()
nb_questions_distinct =len(list(set([s[1]['speaker'] for s in df['Speeche'] if s[1]['sexe']=='F'])))

nb_questions_F = sum([1 for s in df['Speeche'] if s[1]['sexe']=='F']) 
nb_questions_F_distinct =  len(list(set([s[1]['speaker'] for s in df['Speeche'] if s[1]['sexe']=='F'])))
print(nb_questions_F / float(df['Speeche'].count()))
nb_questions_H = df['Speeche'].apply(lambda x: [1 for i, s in enumerate(x)if s['role'] in ['depute_question'] and s['sexe']=='M'])
df['list_interlocuteurs_Unknown'] = df['Speeche'].apply(lambda x: list(set([s['speaker'] for i, s in enumerate(x)if s['role'] not in ['minitre','president','depute_question'] and s['sexe']=='Unknown'])))


268
0.201492537313
